## OpenWeatherMap Python API

In this notebook we will be working with [OpenWeatherMap](https://openweathermap.org/) python API.

OpenWeatherMap is a weather forecasting site, among many others. And they have exposed their data via their API so that we can interact with it using python.

## Get API Key

The first thing we need to do is get an API key. To do that, go [here](https://openweathermap.org/api) and sign up. It should only take a few minutes to do.

After that, scroll down and you see that OpenWeatherMap provides different subscription levels which provide different data. The free API key allows you access to today's weather and the 5 day forcast, among other things. This is what we will be working with today. 

OpenWeatherMap provides really nice documentation for how to submit requests using Python 


The first thing we need to do is install the 'requests' library. Requests is one of many libraries available that someone out there wrote and published for the world to use. We need to use a package manager called conda to install this. 

In [ ]:
!conda install --yes requests

Now let's import the libraries we will be using in this workshop. We don't need to install json because it is already installed with base python.

In [ ]:
import json
import requests

In [ ]:
#Note: this is my API key and is only being used for example purposes. Please don't spam OpenWeatherMap using my API key.
#sign up for your own API key here: https://home.openweathermap.org/users/sign_up
app_id = '333de4e909a5ffe9bfa46f0f89cad105'                    

#Each city in the world has a unique id number. There are over 1,000,000 so I have given you a few to start with.
#You are welcome to look in the data dictionary for more.
city_id_dict = {'Charlottesville': 4752046,                                     
                'New York': 5128581,
                'Chicago': 4887398,
                'Paris': 6455259,
                'Cape Town': 3369157,
                'Beirut': 276781,
                'Dubai': 292223,
                'Shanghai': 1796236,
                'Moscow': 524901,
                'Addis Ababa': 344979,
                'Bangkok': 1609350,
                'Oslo': 6453366,
                'Sao Paulo': 3448439,
                'Bogota': 3688689,
                'Havana': 3553478}


The following is where I create my API request via a URL. As you can see, I create a variable called "request", in which I construct a URL address which will be submitted to OpenWeatherMap. I've already created a couple variables (app_id, city_name, and city_id_string) which I insert into my url request using f-string formatting. 

Below, we are making a request for today's weather. We will do a request for the 5 day forecast later.

In [ ]:
city_name = 'Oslo'                                               #change the city name here
city_id_string = str(city_id_dict[f'{city_name}'])                                         

#Make a request to get today's weather. This is straight from the documentation.
request = requests.get(f'http://api.openweathermap.org/data/2.5/group?APPID={app_id}&id={city_id_string}&units=imperial')               #this actually makes the request to the API via the URL with correct parameters


I now take the request a load the response, which is JSON data. Let's take a look at the weather results below for your chosen city.

In [ ]:
json_data = json.loads(request.text)                                                #json_data is now a dictionary object
print(json_data)



See above that the data returned is simply a dictionary and we can parse that just like we would any other dictionary object.

In [ ]:
for keys, values in json_data.items():
    print(keys)                                             #all the interesting data is contained in the 'list' key


Inside the 'list' key is a list of data. Lists are ordered objects, so we can use indexing to look more closely at each item in the list.

In [ ]:
print(json_data['list'][0])                     #Looking at first object in list


And again, inside that list object we have a dictionary (actually several, nested in eachother), which we can parse the same as before. The actual conditions themselves are in the 'main' key.

In [ ]:
print(json_data['list'][0]['main'])

And one last time, within 'main' is another dictionary of keys and values

In [ ]:
print(json_data['list'][0]['main']['temp_min'])

Or if this gets confusing and unreadable, you can make a variable and store this more cleanly

In [ ]:
min_temp = json_data['list'][0]['main']['temp_min']
print(min_temp)

Now lets make a request to get the 5 day forecast. This returns the forecast at 3 hour intervals for the next 5 days. Again, I took the 'request' straight from the documentation page for OpenWeatherMap

In [ ]:
#make request for 5 day forecast
request = requests.get(f'http://api.openweathermap.org/data/2.5/forecast?id={city_id_string}&APPID=333de4e909a5ffe9bfa46f0f89cad105&units=imperial&')
json_data = json.loads(request.text)

print(json_data)            #note how messy and hard to read this is. Yet, it is a dictionary


This is messy, so let's separate each 3 hour forecast with a blank line. 

There is still a lot of data. How much exactly?  24 hours / 3 hours = 8. 5 days x 8 predictions per day = 40 predictions total.

In [ ]:
#take my word for it, the interesting stuff is in the 'list' key of the json_data dictionary

for measurement in json_data['list']:
    print(measurement)
    print()

Just as an example, let's use string indexing to find one forecast prediction for each day. Luckily, this data is structured so look at the below example. From looking at the data, in the 'dt_txt' key I have a standard date/time stamp. You can see below I am taking the measurement at just 12 noon for each day. The time stamp is at the locations 11-20 in each 'dt_txt' key.

After doing this, I just have 5 results. One for each of the next five days for my target city.

In [ ]:
for daily_measurement in json_data['list']:
    if daily_measurement['dt_txt'][11:20] == '12:00:00':
        print(daily_measurement)
        print(daily_measurement['dt_txt'][:10])
        print(daily_measurement['main']['temp'])                                   #it gets tricky to parse through nested dictionaries
        print()

Here, I use some string formatting and a little more filtering to print some nice looking output. Here, I have found the time stamp of 12 noon for each day. Then within that I have taken the date from the date/time stamp. Then the temperature for each day. And finally put it all together in a string, printing the city, date, and temperature for that day.

In [ ]:
#A cleaner version of the above block
for daily_measurement in json_data['list']:
    time_stamp = daily_measurement['dt_txt'][11:20]
    if time_stamp == '12:00:00':
        day = daily_measurement['dt_txt'][:10]
        temperature = daily_measurement['main']['temp']
        print(f'The temperature in {city_name} on {day} will be {temperature} degrees')

So as you see, this allows you to slice and dice the weather data in many ways. You can automate things, depending on weather conditions, link this to other applications, and so on. I encourange you to play with the script, change the city, look at the results, and so on. 